In [13]:
import plotly

from ctr.reader.reader import Reader
from ctr.resources import *
from ctr.segmentation import BackgroundSubstraction
from ctr.reconstruction import *
import numpy as np
from scipy.io import savemat

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from importlib import reload

In [ ]:
reader = Reader()

# Read original camera images and background
cam1imgs, cam2imgs, backimg = reader.read_images(CAM1_FOLDER, CAM2_FOLDER, BACKG_FOLDER)

# Apply segmentation from part 1 to each set
segmentor = BackgroundSubstraction()
contours_cam1 = segmentor.segmentImgs(cam1imgs, backimg)
contours_cam2 = segmentor.segmentImgs(cam2imgs, backimg)

# Fit polynomials to robot body in order to minimise points to transform, save to csvs
lf = LineFitter()
cam1_lf = lf.fitLines(cam1imgs, 1, contours_cam1, save_folder=LF_RESULTS_FOLDER)
cam2_lf = lf.fitLines(cam2imgs, 2, contours_cam2, save_folder=LF_RESULTS_FOLDER)

# good_fits = [5, 6, 7, 8]
# good_coor_sets = {}
#
# for gf in good_fits:
#     corresp = [cam1_lf[gf],  cam2_lf[gf]]
#     good_coor_sets.update({gf : corresp})

In [3]:
good_coor_sets = {}
for k in cam1_lf.keys():
    corresp = [cam1_lf[k], cam2_lf[k]]
    good_coor_sets.update({k : corresp})

In [4]:
wg = WeightedGraph()
# graph_matches = GRAPH_MATCHES
graph_matches = {}
print('Applying weighted graph...')
# Apply weighted graph matching to sets of coordinates
for idx in good_coor_sets.keys():
    c1 = good_coor_sets.get(idx)[0]
    c2 = good_coor_sets.get(idx)[1]
    wg.set_pairs(c1, c2)
    row_idx, col_idx = wg.solve()
    matching = [row_idx, col_idx]
    graph_matches.update({idx: matching})
    wg.clear()
    print((idx - 4) / len(good_coor_sets.keys()))

Applying weighted graph...
Finding sum assignment...
0.0
Finding sum assignment...
0.06666666666666667
Finding sum assignment...
0.13333333333333333
Finding sum assignment...
0.2
Finding sum assignment...
0.26666666666666666
Finding sum assignment...
0.3333333333333333
Finding sum assignment...
0.4
Finding sum assignment...
0.4666666666666667
Finding sum assignment...
0.5333333333333333
Finding sum assignment...
0.6
Finding sum assignment...
0.6666666666666666
Finding sum assignment...
0.7333333333333333
Finding sum assignment...
0.8
Finding sum assignment...
0.8666666666666667
Finding sum assignment...
0.9333333333333333


In [17]:
import ctr.reconstruction.error as err
reload(err)

<module 'ctr.reconstruction.error' from 'C:\\Users\\Blanca\\Documents\\CTR_Reconstruction\\ctr\\reconstruction\\error.py'>

In [18]:
# print(graph_matches.items())

# rerturns dict of idx : [c1coors, c2coors]
corresp_coors = join_corresp_coors(good_coor_sets, graph_matches)
p1s = []
p2s = []
for i in corresp_coors.keys():
    coors = corresp_coors[i]
    cam1s = good_coor_sets.get(idx)[0]
    cam2s = good_coor_sets.get(idx)[1]
    p1s.append(cam1s)
    p2s.append(cam2s)

hi


In [21]:
# error = error_function(BEST_TRANSF_X, p1s, p2s, mode='mult')
errs, cam2reflect = get_c2reflection(corresp_coors)
show_reflection(cam2imgs, cam2reflect)
robots = transform_many(corresp_coors)
show_robot_3d(robots)

ValueError: cannot reshape array of size 1 into shape (3,4)

In [ ]:
# for key in robots.keys():
#     name = f'lfscipy3_rob{key}.mat'
#     np.savetxt(name, robots[key])



